<a href="https://colab.research.google.com/github/Arshad221b/Smart-Store/blob/master/faceexpression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt

In [0]:
num_classes = 7 #angry, disgust, fear, happy, sad, surprise, neutral
batch_size = 256
epochs = 5

In [0]:
!pip install PyDrive

    100% |████████████████████████████████| 993kB 21.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import pandas as pd

In [0]:
download = drive.CreateFile({'id': '1PMCsXW9ks0GblAddXTzHLPk7X2kDQwm0'})
download.GetContentFile('fer2013.csv')
data = pd.read_csv('fer2013.csv')

In [0]:
print(data)

       emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
5            2  55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...     Training
6            4  20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...     Training
7            3  77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...     Training
8            3  85 84 90 121 101 102 133 153 153 169 177 189 1...     Training
9            2  255 254 255 254 254 179 122 107 95 124 149 150...     Training
10           0  30 24 21 23 25 25 49 67 84 103 120 125 130 139...     Training
11           6  39 75 78 58 58 45 49 48 103 156 81 4

In [0]:
data2 = data['Usage'] == 'Training'
data3 = data['Usage'] == 'PrivateTest'

In [0]:
train = data[data2]

In [0]:
test  = data[data3]

In [0]:
y_train = train['emotion'].values
y_test  = test['emotion'].values

In [0]:
X_train = train['pixels'].values
X_test  = test['pixels'].values

In [0]:

lines = np.array(data)

In [0]:
for i in range(1,28709):
    try:
        emotion, img, usage = lines[i].split(",")
          
        val = img.split(" ")
            
        pixels = np.array(val, 'float32')
        
        emotion = keras.utils.to_categorical(emotion, 7)
    
        if 'Training' in usage:
            y_train.append(emotion)
            x_train.append(pixels)
        elif 'PublicTest' in usage:
            y_test.append(emotion)
            x_test.append(pixels)
    except:
	print("",end="")


(28709,)


In [0]:
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_train = X_train.astype('float32')
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)
X_test = X_test.astype('float32')

ValueError: ignored